In [18]:
!pip3 install python-twitch-client

     |████████████████████████████████| 61 kB 1.7 MB/s eta 0:00:011
  Attempting uninstall: requests
    Found existing installation: requests 2.18.4
    Uninstalling requests-2.18.4:
      Successfully uninstalled requests-2.18.4


In [13]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

In [30]:
from api_keys import client_id
from twitch import TwitchClient
from pprint import pprint


In [24]:
client = TwitchClient(client_id= f'{client_id}')


In [38]:
games = client.games.get_top(50)

In [40]:
pprint(games[35])

{'channels': 329,
 'game': {'box': {'large': 'https://static-cdn.jtvnw.net/ttv-boxart/Teamfight%20Tactics-272x380.jpg',
                  'medium': 'https://static-cdn.jtvnw.net/ttv-boxart/Teamfight%20Tactics-136x190.jpg',
                  'small': 'https://static-cdn.jtvnw.net/ttv-boxart/Teamfight%20Tactics-52x72.jpg',
                  'template': 'https://static-cdn.jtvnw.net/ttv-boxart/Teamfight%20Tactics-{width}x{height}.jpg'},
          'giantbomb_id': 0,
          'id': 513143,
          'locale': 'en-us',
          'localized_name': 'Teamfight Tactics',
          'logo': {'large': 'https://static-cdn.jtvnw.net/ttv-logoart/Teamfight%20Tactics-240x144.jpg',
                   'medium': 'https://static-cdn.jtvnw.net/ttv-logoart/Teamfight%20Tactics-120x72.jpg',
                   'small': 'https://static-cdn.jtvnw.net/ttv-logoart/Teamfight%20Tactics-60x36.jpg',
                   'template': 'https://static-cdn.jtvnw.net/ttv-logoart/Teamfight%20Tactics-{width}x{height}.jpg'},
    